In [1]:
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.4 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 1.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 734.7 kB/s eta 0:00:0000:01


In [2]:
!pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 674.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 19.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 8.8 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 4.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 21.6 MB/s eta 0:00:0000:0100:01


In [3]:
import os
import glob
import torch
import pandas as pd

from tqdm import tqdm
from transformers import pipeline
from torch.utils.data import DataLoader

2023-03-10 16:48:09.630119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 16:48:10.709427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-10 16:48:10.709532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-10 16:48:10.709543: W 

In [45]:
device = torch.device("cuda:0")
hypothesis_template = "This text is about {}."
candidate_labels = ['Politics and Civic Life', 'Community Events and Features', 'Sports', 'Roads and Car Crashes',
                    'Miscellaneous', 'Health', 'Crime', 'Fire and Natural Disasters', 'Weather']

In [46]:
# define dataset class
class LocalNewsDataset():
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]['text']

In [14]:
%%capture
!rm -r categorized_data
!mkdir categorized_data

!mkdir local_data
!mkdir test_data

In [7]:
bart_pipeline = pipeline("zero-shot-classification", model="joeddav/bart-large-mnli-yahoo-answers", device=device)

In [47]:
files = sorted(glob.glob('local_data/*.csv'), key=lambda x: os.path.basename(x).split('_')[0])

In [48]:
for i in range(0, len(files)):
    path = files[i]
    file_name = path.rsplit("/", 1)[-1]
    print(f"Processing file {i}. {file_name}")
    # create an object
    local_news_dataset= LocalNewsDataset(path)
    # create data loader
    dataloader = DataLoader(local_news_dataset, batch_size=4, num_workers=4, shuffle=False)
    # infer the category
    bart_labels, bart_scores = list(), list()
    for item in tqdm(dataloader):
        # infer from model 1
        bart_inferences = bart_pipeline(item, candidate_labels, multi_label=True, hypothesis_template=hypothesis_template)
        for inference in bart_inferences:
            bart_labels.append(inference['labels'][0])
            bart_scores.append(inference['scores'][0])
    # save the results
    df = pd.read_csv(path)
    df['bart_label'] = bart_labels
    df['bart_score'] = bart_scores
    df.to_csv(f"categorized_data/{file_name}", index=False)

Processing file 0. final_springfield.csv


  0%|          | 0/331 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 331/331 [19:46<00:00,  3.58s/it]


Processing file 1. final_stl.csv


  0%|          | 0/424 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 424/424 [31:11<00:00,  4.41s/it]


Processing file 2. final_joplin.csv


  0%|          | 0/137 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 137/137 [08:20<00:00,  3.65s/it]
